In [1]:
import pandas as pd

In [39]:
# display full, non-truncated rows
pd.set_option('display.max_colwidth', -1)

columns = ['class', 'unsure', 'area_code?',
           'date', 'camera', 'satellite',
           'deforested_area', 'unknown',
           'area', 'state', 'abbreviation',
           'lat', 'long']

brasil_data = pd.read_csv('deter_public_amz_deter_public.csv', header=None, names=columns)

<ipython-input-39-59e4d376abf0>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
d:\csfiles\lambda\git\planethack2020\venv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (2,6,8,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [40]:
brasil_data

,class,unsure,area_code?,date,camera,satellite,deforested_area,unknown,area,state,abbreviation,lat,long
0,CICATRIZ_DE_QUEIMADA,NaN,165105,7/27/2020,AWFI,CBERS-4,0.0000000000,NaN,0.0752305339,Novo Repartimento,PA,-50.47120000000,-4.91754000000
1,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.0000000000,NaN,0.3442344303,Altamira,PA,-55.05660000000,-6.91208000000
2,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.0000000000,NaN,4.7885689019,Brasil Novo,PA,-52.57660000000,-3.56411000000
3,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.0000000000,NaN,1.0676025778,Altamira,PA,-55.20420000000,-6.78340000000
4,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.0000000000,NaN,0.0676800448,Alenquer,PA,-54.97660000000,-1.79718000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186409,MINERACAO,NaN,210124,6/24/2020,WFI,CBERS-4A,0,NaN,0.110526,Cumaru do Norte,PA,-50.9172,-7.76491
186410,MINERACAO,NaN,165105,7/27/2020,AWFI,CBERS-4,0,NaN,0.0956882,Bannach,PA,-50.8812,-7.41031
186411,MINERACAO,NaN,165105,7/27/2020,AWFI,CBERS-4,0,NaN,0.063135,Bannach,PA,-50.8652,-7.38636
186412,MINERACAO,NaN,169105,1/14/2018,AWFI,CBERS-4,0.121459,├üREA DE PROTE├ç├âO AMBIENTAL DO TAPAJ├ôS,0.121459,Jacareacanga,PA,-57.4651,-6.16502


In [41]:
print(brasil_data['lat'][0])
print(brasil_data['long'][0])

-50.47120000000
-4.91754000000


In [44]:
brasil_data['base_url'] = brasil_data.apply(lambda row: f"https://www.planet.com/explorer/#/center/{row['lat']:.8},{row['long']:.8}/zoom/8", axis=1)

In [45]:
brasil_data['base_url']

0         https://www.planet.com/explorer/#/center/-50.4712,-4.91754/zoom/8
1         https://www.planet.com/explorer/#/center/-55.0566,-6.91208/zoom/8
2         https://www.planet.com/explorer/#/center/-52.5766,-3.56411/zoom/8
3         https://www.planet.com/explorer/#/center/-55.2042,-6.78340/zoom/8
4         https://www.planet.com/explorer/#/center/-54.9766,-1.79718/zoom/8
                                        ...                                
186409    https://www.planet.com/explorer/#/center/-50.9172,-7.76491/zoom/8
186410    https://www.planet.com/explorer/#/center/-50.8812,-7.41031/zoom/8
186411    https://www.planet.com/explorer/#/center/-50.8652,-7.38636/zoom/8
186412    https://www.planet.com/explorer/#/center/-57.4651,-6.16502/zoom/8
186413    https://www.planet.com/explorer/#/center/-57.3608,-5.73095/zoom/8
Name: base_url, Length: 186414, dtype: object